 # **<span style="font-size:larger;"> 01: <span style="color:blue">Data Collection & Cleaning</span>** #

This project initially used just one dataset of 96,005 URLs-- about 50% legitimate URLs and 50% phishing URLs. While the model created was able to perform with an 91% accuracy on the testing data, model deployment seemed to have its own pitfalls. Simple websites such as www.google.com were classified as phishing. When taking a closer look at our dataset, it was evident that legitimate URL samples did not include short, simple URLs. An additional dataset is merged with the original to improve our model upon deployment.

#### Imports and Read-Ins:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Original dataset
df = pd.read_csv("../data/url_original.csv")

In [68]:
# Additional dataset
df2 = pd.read_csv("../data/url_additional.csv")

# I. Initial Read

In [35]:
df.head()

,domain,label
0,nobell.it/70ffb52d079109dca5664cce6f317373782/...,1.0
1,www.dghjdgf.com/paypal.co.uk/cycgi-bin/webscrc...,1.0
2,serviciosbys.com/paypal.cgi.bin.get-into.herf....,1.0
3,mail.printakid.com/www.online.americanexpress....,1.0
4,thewhiskeydregs.com/wp-content/themes/widescre...,1.0


In [36]:
df2.head()

,Unnamed: 0,url,label,result
0,0,https://www.google.com,benign,0
1,1,https://www.youtube.com,benign,0
2,2,https://www.facebook.com,benign,0
3,3,https://www.baidu.com,benign,0
4,4,https://www.wikipedia.org,benign,0


In [37]:
def initial_read(df):
    print('SHAPE')
    print(df.shape)
    print('=====')
    print('DTYPES')
    print(df.dtypes)
    print('=====')
    print('NULL VALUES')
    print(df.isnull().sum())
    print('=====')

In [38]:
initial_read(df)

SHAPE
(96005, 2)
=====
DTYPES
domain     object
label     float64
dtype: object
=====
NULL VALUES
domain     0
label     92
dtype: int64
=====


In [39]:
initial_read(df2)

SHAPE
(450176, 4)
=====
DTYPES
Unnamed: 0     int64
url           object
label         object
result         int64
dtype: object
=====
NULL VALUES
Unnamed: 0    0
url           0
label         0
result        0
dtype: int64
=====


# II. Initial Cleaning

### Dropping Null Values

From the above function, we can see that there were 92 URLs without a label (phishing or legitimate URL). We will simply drop these rows. By checking the shape of the dataframe, we can ensure that 92 null values have been dropped.

In [40]:
df.dropna(inplace=True)

In [41]:
# Quick check
df.isnull().sum()

domain    0
label     0
dtype: int64

### Dropping Columns

In [42]:
df2.drop(columns = ['Unnamed: 0', 'label'], inplace=True)

### Changing Dtypes

Dtypes are changed so the two datasets match.

In [43]:
df['label'] = df['label'].astype(int)

In [44]:
df.dtypes

domain    object
label      int64
dtype: object

### Renaming Columns

Columns are renamed so the two datasets match.

In [45]:
df.rename(columns={"domain": "url", "label": "phishing"}, inplace=True)
df2.rename(columns={"result": "phishing"}, inplace=True)

### Adding Protocol

The original dataset does not include a protocol (such as 'http://') in the provided URLs. In order for a future use of urlparse to work efficiently on the concatenated DataFrame, all URLs must include a protocol. It is important to note that features extracted from the protocol will not be used in the model, but simply aid in the split of different URL parts.

In [46]:
df['url'] = 'https://' + df['url'].astype(str)

### Confirming Changes

Final confirmation that all necessary changes were made:

In [47]:
df.head()

,url,phishing
0,https://nobell.it/70ffb52d079109dca5664cce6f31...,1
1,https://www.dghjdgf.com/paypal.co.uk/cycgi-bin...,1
2,https://serviciosbys.com/paypal.cgi.bin.get-in...,1
3,https://mail.printakid.com/www.online.american...,1
4,https://thewhiskeydregs.com/wp-content/themes/...,1


In [48]:
df2.head()

,url,phishing
0,https://www.google.com,0
1,https://www.youtube.com,0
2,https://www.facebook.com,0
3,https://www.baidu.com,0
4,https://www.wikipedia.org,0


# III. Final Changes

### Concatenating DataFrames

In [59]:
df_final = pd.concat([df, df2])

In [60]:
initial_read(df_final)

SHAPE
(546089, 2)
=====
DTYPES
url         object
phishing     int64
dtype: object
=====
NULL VALUES
url         0
phishing    0
dtype: int64
=====


### Dropping Duplicates

We can see there were 194 duplicated URLs. These will be dropped.

In [61]:
df_final[df_final.duplicated()]

,url,phishing
35411,https://bin/webscr?cmd=_login-,1
45182,https://'www.allegropl.xaa.pl/enter_login.html...,1
129252,https://www.airnav.com/airports/us/TX,0
140534,https://www.apple.com/quicktime/,0
167423,https://www.conitec.net/english/gstudio/,0
...,...,...
345307,https://homepage.ntlworld.com/fetchfido2/inter...,0
345310,https://www.infopeople.org/search/chart.html,0
345313,https://l-lists.com/en/lists/lv2azk.html,0
345318,https://www.bruceclay.com/searchenginechart.pdf,0


In [63]:
df_final.drop_duplicates(inplace=True)

# IV. Saving DataFrame

In [65]:
df_final.to_csv('../data/url_concatenated.csv', index=False)

# ***Next Notebook*** - [02: Pre-Processing]('https://github.com/ksylvia16/Phishing-URL-Detection/blob/40cd1db7e428eaeeb979e9a84e5561856defde76/code/02_Pre-Processing.ipynb')